In [1]:
from utilities import *
from astropy.io import fits
from astropy.table import QTable
import glob
import radio_beam
import sys

source='DSiv'
fnum=fields[source]

linemodelhome='/blue/adamginsburg/d.jeff/SgrB2DS_COMAnalysis/linemodels/'
linemodelversion='firstrelease'
linemodelpath=linemodelhome+f'{linemodelversion}/{source}/'

targetmolecule='C2H5OH'
targetmoleculepath=linemodelpath+f'{targetmolecule}.fits'
targetmoltable=QTable.read(targetmoleculepath)

othermoleculepaths=glob.glob(linemodelpath+'*.fits')
othermoleculepaths.remove(targetmoleculepath)

ch3ohpath=ch3oh_sourcedict[source]
targetpix=pixdict[source]
origsourcepath=f'/blue/adamginsburg/d.jeff/SgrB2DSreorg/field{fnum}/CH3OH/{source}{ch3ohpath}'
contpath=origsourcepath+'reprojectedcontinuum.fits'

fwhmpath=glob.glob(origsourcepath+'*fwhm*')[0]
fwhmmap=fits.getdata(fwhmpath)*u.km/u.s
fwhm_at_pix=fwhmmap[targetpix[0],targetpix[1]]

reprojcontfits=fits.open(contpath)
reprojcont=reprojcontfits[0].data*u.Jy
reprojcontrestfreq=225*u.GHz#manual addition 11/9/2022, wiggle room w/i GHz
cntmbeam=radio_beam.Beam.from_fits_header(reprojcontfits[0].header)
reprojcont_K=reprojcont.to('K',cntmbeam.jtok_equiv(reprojcontrestfreq))
continuumlevel=reprojcont_K[targetpix[0],targetpix[1]]

In [2]:
safelines=[]
minimumcontaminantthreshold=0.1#If contaminant lines have more than this times the target line's flux, the target line is considered contaminated and removed
firstline=True
for targetline in targetmoltable:
    flux=targetline['ModelBrightness']
    reffreq=targetline['ReferenceFrequency']
    qn=targetline['QNs']
    if flux <= continuumlevel:
        continue
    else:
        possiblecontaminants={}
        for tablepath in othermoleculepaths:
            othermolecule=tablepath.replace(f'{linemodelpath}','')
            othermolecule=othermolecule.replace('.fits','')
            
            othermoltable=QTable.read(tablepath)
            fwhm_Hz=velocitytofreq(fwhm_at_pix,reffreq)
            diff=np.abs(othermoltable['ReferenceFrequency']-reffreq)
            findcontam=np.where(diff<=fwhm_Hz)[0]
            if len(findcontam) > 0:
                candidate_contaminant_lines=othermoltable[findcontam]
                numcontaminants=len(candidate_contaminant_lines)
                if numcontaminants > 1:
                    contaminantflux=np.sum(candidate_contaminant_lines['ModelBrightness'])
                elif numcontaminants == 1:
                    contaminantflux=candidate_contaminant_lines['ModelBrightness']
                if contaminantflux >= (flux*minimumcontaminantthreshold):
                    print(f'{targetmolecule}\'s {qn} @ {reffreq} likely contaminated by {numcontaminants} {othermolecule} lines')
                    possiblecontaminants.update({othermolecule:candidate_contaminant_lines})
        if len(possiblecontaminants) == 0:
            if firstline:
                safelines=QTable(targetline)
                firstline=False
            else:
                safelines.add_row(targetline)

C2H5OH's 13(0,13)(1)-12(1,12)(1) @ 217.22271295402936 GHz likely contaminated by 2 CH3OCHO lines
C2H5OH's 13(1,13)(1)-12(0,12)(1) @ 217.45703324860926 GHz likely contaminated by 3 CH2CHCN lines
C2H5OH's 30(3,27)(2)-30(2,28)(2) @ 219.13379755440454 GHz likely contaminated by 1 HC3Nv7_1 lines
C2H5OH's 19(5,15)(2)-19(4,16)(2) @ 231.69539437409216 GHz likely contaminated by 1 CH3OCHO lines
C2H5OH's 22(5,18)(2)-22(4,19)(2) @ 231.7478202193596 GHz likely contaminated by 1 CH3NCO,vb=0 lines
C2H5OH's 22(5,18)(2)-22(4,19)(2) @ 231.7478202193596 GHz likely contaminated by 4 CH3OCH3 lines
C2H5OH's 15(5,10)(2)-15(4,11)(2) @ 232.0335568431936 GHz likely contaminated by 1 13CH3CN lines
C2H5OH's 14(1,14)(1)-13(1,13)(1) @ 232.5541846575604 GHz likely contaminated by 1 CH3OCHO lines
C2H5OH's 16(5,12)(2)-16(4,13)(2) @ 232.76644027347945 GHz likely contaminated by 1 CH3CHO lines
C2H5OH's 24(5,20)(2)-24(4,21)(2) @ 233.16604694423188 GHz likely contaminated by 4 CH3CH2CN lines
C2H5OH's 24(5,20)(2)-24(4,21)

In [3]:
doublets=[]
for targetline in safelines:
    flux=targetline['ModelBrightness']
    reffreq=targetline['ReferenceFrequency']
    qn=targetline['QNs']
    fwhm_Hz=velocitytofreq(fwhm_at_pix,reffreq)
    diff=np.abs(targetline['ReferenceFrequency']-reffreq)
    findcontam=np.where(diff<=fwhm_Hz)[0]
    if len(findcontam) > 1:
        print(f'{qn} has {len(findcontam)} companion lines')
        doublets.append(safelines[findcontam])
if len(doublets) == 0:
    print(f'{targetmolecule} has no doublets')

C2H5OH has no doublets


Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.


In [4]:
fwhm_Hz

<Quantity 4.90835077 MHz>

In [5]:
safelinepathbase=linemodelpath+'safelines/'
safelinepath=safelinepathbase+f'{targetmolecule}.fits'
if not os.path.exists(safelinepath):
    os.mkdir(safelinepathbase)
safelines.write(safelinepath)

In [6]:
safelines

QNs,RestFrequency,ReferenceFrequency,Eupper,Degeneracy,Aij,ModelBrightness,ModelNupper
,GHz,GHz,K,,Hz,K,1 / cm2
bytes25,float64,float64,float64,int64,float64,float64,float64
"21(5,16)(2)-21(4,17)(2)",218.5545074,218.51468258955057,226.01385361044416,43,7.18477867162649e-05,24.487710620483377,303410550089506.9
"35(4,31)(2)-35(3,32)(2)",218.9432887,218.90339304616344,559.6887547607863,71,7.382877809861699e-05,18.07170580526624,216819483177813.22
"13(2,11)(0)-12(2,10)(0)",230.6725581,230.63052514899155,138.6199773674354,27,0.00010602668351003007,25.33782430604667,237240956068412.0
"16(5,11)(2)-16(4,12)(2)",230.9537832,230.91169900440076,145.77872777262775,33,7.765111093374898e-05,22.312749757235842,284797679681723.7
"14(0,14)(2)-13(1,13)(2)",230.9913834,230.9492923529253,85.52671483307333,29,0.00011962150933731016,34.54872178079723,291137534987727.8
"41(5,36)(2)-41(4,37)(2)",230.9966489,230.95455689345053,767.3782930658575,83,9.578999680115902e-05,14.687355768419451,150496397648614.94
"21(5,17)(2)-21(4,18)(2)",231.5585678,231.51637340108192,225.95122187481488,43,8.357364920231331e-05,25.35111441741706,303458250410051.2
"20(5,16)(2)-20(4,17)(2)",231.5609215,231.51872667219277,208.21222307878986,41,8.265677678007207e-05,25.006365380615577,302517667473578.6


In [48]:
reffreq

<Quantity 217.45610974 GHz>

MaskedNDArray(True)